In [28]:
from pathlib import Path
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from langgraph.graph import MessagesState, StateGraph
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import END
from langchain_core.documents import Document
import time
import io
from contextlib import redirect_stdout
from datetime import datetime
from pathlib import Path
import json
from typing import List, Dict, Any
import uuid
from qdrant_client import QdrantClient, models
import os
from dotenv import load_dotenv
from retrieval import retrieve, set_collection
from langchain_google_genai import ChatGoogleGenerativeAI
from qdrant_client.http.models import Filter, FieldCondition, MatchAny
import re

# load in environment variables
env_path = "../../.env"
load_dotenv(dotenv_path=env_path)
QDRANT_CONNECT = os.getenv("QDRANT_CONNECT")
COLLECTION_NAME = os.getenv("DOCUMENT_COLLECTION")
GOOGLE_API = os.getenv("GOOGLE_API_KEY")

try:
    qdrant_client = QdrantClient(url=QDRANT_CONNECT)
except Exception as e:
    print(f"Error connecting to Qdrant: {e}")

set_collection(qdrant_client)

# Provided retrieve tool for querying DB, Search Engine Team will write code replacing
# this to allow for query expansion
def set_api_key(api_key_variable: Path):
    os.environ["GOOGLE_API_KEY"] = api_key_variable

def initialize_llm(gemini_model = 'gemini-2.0-flash'):
    set_api_key(GOOGLE_API)
    llm = ChatGoogleGenerativeAI(model=gemini_model, max_tokens=None)
    return llm

llm = initialize_llm()
combine_llm = initialize_llm()

In [ ]:
schema = {
    "type": "object",
    "properties": {
        "subquery":{
            "type": "string",
            "description": "A specific subquery derived from the user query for efficient document retrieval.",
            "examples": "Describe key aspects of Pacific Gas and Electric's (PG&E) 2023 General Rate Case application."
        },
        "search_strings":{
            "type": "array",
            "items": {
                "type": "string",
                "description": "A search string derived from the subquery that will be used to retrieve relevant documents from the vector database. We include multiple in case the subquery involves multiple aspects or entities.",
            },
            "description": "A list of search strings that will be used to retrieve relevant documents from the vector database.",
            "maxItems": 2,
            "minItems": 1
        },
        "proceeding_id":{
            "type": "array",
            "items":{
                "type": "string",
                "description": "The unique identifier for the proceeding related to the subquery extracted from the subquery if available.",
                "examples": ["A.23-03-005", "A.21-06-021"]
            },
            "description": "List of unique identifiers for the proceeding related to the subquery. This field should only be populated if the id of the proceeding is clearly stated in the original user query.",
        }
    }
}

PROMPT = f"""
You are a tool in a retrieval-augmented generation (RAG) system. Your job is to decompose a complex user query into specific and focused subqueries, each paired with relevant search strings. These subqueries will be used to retrieve relevant documents from a vector database.

Important:

- DO NOT answer the user query.

- You are NOT allowed to summarize, explain, or discuss the topic.

- Only return a list of JSON objects in the schema format below.

- Each subquery must be a JSON object with the following fields:

- Only add to the proceeding_id metadata field if the id of the proceeding is CLEARLY STATED in the original user query.

- If there is potential for multiple proceedings, do not include the proceeding_id field in the JSON object.

Schema:
{schema}

Only return a list of such JSON objects, without any extra commentary, explanation, or markdown formatting.

"""

In [3]:
from langchain_core.documents import Document
from langchain_core.tools import tool
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer

from advanced_retrieval import query_db, crossEncoderQuery, hydeRetrieval, hydeCrossEncoderRetrieval

import os
from dotenv import load_dotenv

load_dotenv()

K = 8
DOCUMENT_COLLECTION = os.getenv("DOCUMENT_COLLECTION")  # Placeholder for the ChromaDB collection

embedding_model_name = os.getenv("EMBEDDING_MODEL", "all-MiniLM-L6-v2")
embedding_function = SentenceTransformer(embedding_model_name)

In [ ]:
# Multithreading for api calls
import asyncio
import time
from tenacity import AsyncRetrying, stop_after_attempt, RetryError, wait_exponential, retry
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage


def retrieve_context(query: str, k: int = 8, search_filter: Filter = None) -> str:
    """Retrieve information related to a query."""
    # Query qdrant directly
    if not qdrant_client:
        raise ValueError("Qdrant client is not initialized. Please set the QDRANT_CONNECT environment variable.")
    
    results = crossEncoderQuery(
        qdrant_client=qdrant_client,
        query=query,
        collection_name=DOCUMENT_COLLECTION,
        k=k,
        search_filter = search_filter 
    )

    # Format results for LangChain compatibility
    retrieved_docs = []
    for result in results:
        doc_id = result.payload['document_id']
        content = result.payload['text']
        metadata = {k: v for k, v in result.payload.items() if k != 'text'} if result.payload else {}

        doc = Document(page_content=content, metadata=metadata)
        retrieved_docs.append(doc)

    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content} Document Link: {doc.metadata.get('source_url', 'N/A')}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs



def getFormattedQuery(subquery: Dict[str, Any]):
    search_strings = subquery.get("search_strings", [])
    proceeding_ids = subquery.get("proceeding_id", None) # list of potential IDs to search through
    subquery_text = subquery.get("subquery", "")

    for i,proceeding in enumerate(proceeding_ids):
        # we need to format it so it can be used in the query, remove everything but the alphanumeric characters
        proceeding_ids[i] = re.sub(r'[^A-Za-z0-9]', '', proceeding)# Remove non-alphanumeric characters
        print(f"Proceeding ID {i}: {proceeding_ids[i]}")
        

    # create filter for proceeding_id if it exists
    query_filter = None
    if proceeding_ids:
        query_filter = Filter(
            must=[
                models.FieldCondition(
                    key="proceeding_id",
                    match=models.MatchAny(any=proceeding_ids)
                )
            ]
        )

    search_result = retrieve_context(
        query=search_strings[0], # use first string for retrieval, will modify later to use all and combine
        k=K,
        search_filter = query_filter
    )

    formatted_query = f"""
    You are "GRC Regulatory Analysis Expert," an AI assistant specialized in California GRC proceedings.
                </SYSTEM>

                <INFORMATION SOURCES>
                Base your responses EXCLUSIVELY on:
                1. Retrieved documents (HIGHEST PRIORITY)
                2. User-provided context in the current session

                The retrieval system has already provided you with the most relevant information.
                Always cite your sources with specific references (e.g., "PG&E 2023 GRC, Exhibit 4, p.15").
                </INFORMATION SOURCES>

                <IDENTITY AND EXPERTISE>
                You are a regulatory specialist focused exclusively on California General Rate Case (GRC) proceedings and related CPUC filings with expertise in:
                - Rate case applications and testimony
                - Revenue requirement analysis
                - Procedural requirements and timelines
                - CPUC decisions and precedents
                </IDENTITY AND EXPERTISE>

                <RESPONSE FORMAT>
                Structure your responses with:
                1. Concise summary of key findings
                2. Detailed analysis with multiple supporting citations
                3. Relevant regulatory background and historical context
                4. Discussion of practical implications
                5. Throughout your response, for example after each bullet point, supply a list of markdown links to the sources you used to generate that part of the response as a comma seperated list.
                

                Use markdown formatting (headers, tables, bullets) to enhance readability.
                </RESPONSE FORMAT>

                <PROFESSIONAL TONE>
                Maintain a voice that is:
                - Authoritative yet accessible
                - Technically precise
                - Thorough and explanatory
                - Objective in regulatory interpretation
                </PROFESSIONAL TONE>

                <ACCURACY REQUIREMENTS>
                - Never invent citations, docket numbers, or proceedings
                - Clearly indicate when information is missing or insufficient
                - Present multiple interpretations when guidance is ambiguous
                - Quote directly from sources for critical regulatory language
                </ACCURACY REQUIREMENTS>

                <SCOPE LIMITATIONS>
                Address only topics related to California GRC proceedings and CPUC regulatory matters.
                For other topics, politely explain they fall outside your expertise.
                </SCOPE LIMITATIONS>

                Always end responses with: "Would you like me to explore any aspect of this response in greater depth or address related regulatory considerations?"

                Document Context: {search_result[0]}

                User Query: {subquery_text}
                """

    return formatted_query

async def process_subqueries(original_query:str, subqueries: str):
    """
    Process the subqueries to ensure they are in the correct format.
    """
    # Manually cleaning the response since gemini does not seem to be able to
    cleaned_response = subqueries.strip()
    if cleaned_response.startswith("```json"):
        cleaned_response = cleaned_response.lstrip("```json").strip()
    if cleaned_response.endswith("```"):
        cleaned_response = cleaned_response.rstrip("```").strip()
    print(cleaned_response)
    
    try:
        response_json = json.loads(cleaned_response)
    except json.JSONDecodeError:
        print("Failed to parse response as JSON. Here's the raw content:")
        return None
    
    combined_responses = ''
    queries = []

    for i,item in enumerate(response_json):
        queries.append(
            {
            'query': item['subquery'],
            'index': i,
            'prompt': getFormattedQuery(item)
            }
        )
    
    combined_queries =  await multiThreadedQueries(queries)
    formatted_return = '\n\n'.join(combined_queries)
    answer = await combineSubqueries(original_query, formatted_return)

    return answer

async def combineSubqueries(original_query:str, formatted_answers: str):
    combine_queries_prompt = f"""
    You are an expert in regulatory analysis, tasked with combining multiple subqueries into a single, coherent response.
    
    Please synthesize the responses to all previous subqueries into a comprehensive analysis of the California GRC proceedings, ensuring all relevant details are integrated and presented coherently. Focus on providing a unified understanding of the regulatory landscape and its implications.

    Ensure that the final response is well-structured, and maintains the same level of detail and professionalism as the individual subquery responses. Do your best to keep the responses and thorough as possible, maintaining enough details from each subquery answer.

    To make the response more seamless, act as though this is a single response to the original user query, and not a response to multiple subqueries.

    Maintain formatting such as bullets, headers, and markdown links from the original subqueries. Do not say "The provided documents" in the response, instead refer to context as "availible data".

    ORIGINAL USER QUERY:
    {original_query}
    
    SUBQUERIES AND RESPONSES:
    
    """

    final_prompt = combine_queries_prompt + formatted_answers
    messages = [HumanMessage(content=final_prompt)]
    results = combine_llm.invoke(messages)
    return results.content if isinstance(results, AIMessage) else "Failed to synthesize subqueries."
    


@retry(
    wait=wait_exponential(multiplier=1, min=4, max=10),
    stop=stop_after_attempt(3)
)
async def asyncQueryLLM(query: Dict[str, Any]) -> str:
    prompt = query.get('prompt', "")

    if not prompt:
        raise ValueError("Prompt is empty or not provided in the query dictionary.")

    try:
        messages = [HumanMessage(content=prompt)]
        response = await llm.ainvoke(messages)
        llm_response = response.content if isinstance(response, AIMessage) else "Failed to retrieve response for Subquery\n"
        
        formatted_response = f"""
        Subquery {query['index'] + 1}:\n{query["query"]}\nGenerated Response:\n{llm_response}\n
        """
        return formatted_response

    except Exception as e:
        print(f"Error querying LLM: {e}")
        raise e

async def multiThreadedQueries(queries: List[str]):
    tasks = []
    for query in queries:
        tasks.append(asyncQueryLLM(query))
    
    results = []

    for future in asyncio.as_completed(tasks):
        try:
            result = await future
            results.append(result)
        except RetryError as e:
            print(f"RetryError in async task: {e}")
            results.append("Error processing subquery after retries.")
        except Exception as e:
            print(f"Error in async task: {e}")
            results.append("Error processing subquery.")

    # sort by query index
    results.sort(key=lambda x: int(re.search(r'Subquery (\d+):', x).group(1)) - 1)
    return results


In [5]:
test_query = "Based on the public record in proceeding A.21-06-021, analyze the differences between PG&E's original undergrounding proposal and what was authorized in Decision 23-11-069. Include: (1) comparison of the miles of undergrounding requested by PG&E versus what was approved by the CPUC and the stated rationale for the reduction in the decision, (2) analysis of the Administrative Law Judge's Proposed Decision versus the Alternate Proposed Decision regarding undergrounding miles and which approach was ultimately adopted, (3) summary of intervenor positions on undergrounding from their filed testimony, including any opposition or alternative proposals, (4) identification of the specific CPUC findings regarding cost-effectiveness of undergrounding versus covered conductor alternatives as stated in the final decision, and (5) explanation of how the decision addresses PG&E's 10,000-mile undergrounding program timeline based on the proceeding record."

combined_prompt = PROMPT + f"\nUSER QUERY:\n{test_query}"

print("Combined Prompt:")
print(combined_prompt)

response = llm.invoke(combined_prompt)

print("Response from LLM:")
print(response.content)



Combined Prompt:

You are a tool in a retrieval-augmented generation (RAG) system. Your job is to decompose a complex user query into specific and focused subqueries, each paired with relevant search strings. These subqueries will be used to retrieve relevant documents from a vector database.

Important:

- DO NOT answer the user query.

- You are NOT allowed to summarize, explain, or discuss the topic.

- Only return a list of JSON objects in the schema format below.

- Each subquery must be a JSON object with the following fields:

- Only add to the proceeding_id metadata field if the id of the proceeding is CLEARLY STATED in the original user query.

Schema:
{'type': 'object', 'properties': {'subquery': {'type': 'string', 'description': 'A specific subquery derived from the user query for efficient document retrieval.', 'examples': "Describe key aspects of Pacific Gas and Electric's (PG&E) 2023 General Rate Case application."}, 'search_strings': {'type': 'array', 'items': {'type': '

In [34]:
await process_subqueries(test_query, response.content)

[
  {
    "subquery": "Compare PG&E's original undergrounding proposal in A.21-06-021 with the authorized undergrounding in Decision 23-11-069, focusing on the difference in miles and the CPUC's rationale for any reduction.",
    "search_strings": [
      "A.21-06-021 PG&E undergrounding proposal vs Decision 23-11-069",
      "Decision 23-11-069 rationale for undergrounding reduction"
    ],
    "proceeding_id": [
      "A.21-06-021"
    ]
  },
  {
    "subquery": "Analyze the Administrative Law Judge's Proposed Decision (PD) versus the Alternate Proposed Decision (APD) in A.21-06-021 regarding undergrounding miles, and identify which approach was adopted in Decision 23-11-069.",
    "search_strings": [
      "A.21-06-021 ALJ PD vs APD undergrounding miles",
      "Decision 23-11-069 undergrounding miles PD APD"
    ],
    "proceeding_id": [
      "A.21-06-021"
    ]
  },
  {
    "subquery": "Summarize intervenor positions on undergrounding in A.21-06-021, including opposition and alte

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Proceeding ID 0: A2106021


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Proceeding ID 0: A2106021


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Proceeding ID 0: A2106021


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

## Comprehensive Analysis of PG&E's Undergrounding Proposal in A.21-06-021 and Decision 23-11-069

This analysis synthesizes the public record in proceeding A.21-06-021 and Decision 23-11-069 to provide a comprehensive understanding of the regulatory landscape surrounding PG&E's undergrounding proposal. It encompasses the evolution of the proposal, the CPUC's rationale for modifications, a comparative analysis of the Proposed Decision (PD) and Alternate Proposed Decision (APD), and considerations of cost-effectiveness and timelines.

### 1. Comparison of Undergrounding Miles: PG&E's Proposal vs. CPUC Authorization

*   **Initial Proposal:** PG&E initially proposed undergrounding **3,600 miles** of distribution lines, announced in February 2022, as part of its broader **10,000-mile** undergrounding concept announced in late 2021. [[A.21-06-021, Proposed Decision, p. 249](https://docs.cpuc.ca.gov/PublishedDocs/Efile/G000/M520/K418/520418881.PDF), [A.21-06-021, Alternate, p. 248](https://

"## Comprehensive Analysis of PG&E's Undergrounding Proposal in A.21-06-021 and Decision 23-11-069\n\nThis analysis synthesizes the public record in proceeding A.21-06-021 and Decision 23-11-069 to provide a comprehensive understanding of the regulatory landscape surrounding PG&E's undergrounding proposal. It encompasses the evolution of the proposal, the CPUC's rationale for modifications, a comparative analysis of the Proposed Decision (PD) and Alternate Proposed Decision (APD), and considerations of cost-effectiveness and timelines.\n\n### 1. Comparison of Undergrounding Miles: PG&E's Proposal vs. CPUC Authorization\n\n*   **Initial Proposal:** PG&E initially proposed undergrounding **3,600 miles** of distribution lines, announced in February 2022, as part of its broader **10,000-mile** undergrounding concept announced in late 2021. [[A.21-06-021, Proposed Decision, p. 249](https://docs.cpuc.ca.gov/PublishedDocs/Efile/G000/M520/K418/520418881.PDF), [A.21-06-021, Alternate, p. 248](h

In [14]:
query = 'Summarize intervenor positions on undergrounding in A.21-06-021, including opposition and alternative proposals from filed testimony.'

query_filter = Filter(
            must=[
                models.FieldCondition(
                    key="proceeding_id",
                    match=models.MatchAny(any=["A2106021"])
                )
            ]
        )

results = query_db(
        query=query, # use first string for retrieval, will modify later to use all and combine
        k=50,
        search_filter = query_filter,
        qdrant_client=qdrant_client,
        collection_name=DOCUMENT_COLLECTION,
    )

# Format results for LangChain compatibility
retrieved_docs = []
for result in results:
    doc_id = result.payload['document_id']
    content = result.payload['text']
    metadata = {k: v for k, v in result.payload.items() if k != 'text'} if result.payload else {}

    doc = Document(page_content=content, metadata=metadata)
    retrieved_docs.append(doc)

    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content} Document Link: {doc.metadata.get('source_url', 'N/A')}")
        for doc in retrieved_docs
    )
print(serialized)



Source: {'chunk_index': 11, 'document_id': '418717765', 'proceeding_id': 'A2101003', 'source_url': 'https://docs.cpuc.ca.gov/PublishedDocs/Efile/G000/M418/K717/418717765.PDF', 'published_date': '11/01/2021', 'year': 0, 'title': 'Motion filed by WATER RATE ADVOCATES FOR TRANSPARENCY, EQUITY AND SUSTAINABILITY on 10/26/2021 Conf# 173304', 'doc_type': 'Motion'}
Content: May 27, 2021; Motion for an extension to serve intervenor testimony. 
12 D.0606066 with modifications 
13 GO 66-D 3.3 and 3.4 Document Link: https://docs.cpuc.ca.gov/PublishedDocs/Efile/G000/M418/K717/418717765.PDF

Source: {'chunk_index': 13, 'document_id': '397312194', 'proceeding_id': 'A2107003', 'source_url': 'https://docs.cpuc.ca.gov/PublishedDocs/Efile/G000/M397/K312/397312194.PDF', 'published_date': '08/09/2021', 'year': 0, 'title': 'Protest filed by Cal Advocates/Zarchy/CPUC on 08/06/2021 Conf# 169341', 'doc_type': 'Protest'}
Content: 01/24/22 
Other Intervenor Testimony 
10/06/21 
02/07/22 
Rebuttal Testimony 
10/

In [6]:
def create_proceeding_id_index():
    """Create a keyword index on proceeding_id field"""
    try:
        print("Creating index on proceeding_id field...")
        
        qdrant_client.create_payload_index(
            collection_name=DOCUMENT_COLLECTION,
            field_name="proceeding_id",
            field_schema="keyword"  # Best for exact string matches like IDs
        )
        
        print("✅ Index creation request submitted successfully!")
        print("⏳ Index is being built in the background...")
        
        return True
        
    except Exception as e:
        print(f"❌ Index creation failed: {e}")
        return False

# Create the index
create_proceeding_id_index()

Creating index on proceeding_id field...
✅ Index creation request submitted successfully!
⏳ Index is being built in the background...


True